In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

labels = ["Lucky Star 1", "Lucky Star 2", "Ball 1", "Ball 2", "Ball 3", "Ball 4", "Ball 5"]
column_labels = ["Lucky Star 1", "Lucky Star 2", "Ball 1", "Ball 2", "Ball 3", "Ball 4", "Ball 5", "Ball Delta", "Ball Y Delta", "LS Delta", "LS Y Delta",
    "Ball 1 Repeat Last 5", "Ball 2 Repeat Last 5", "Ball 3 Repeat Last 5", "Ball 4 Repeat Last 5", "Ball 5 Repeat Last 5", "Lucky Star 1 Repeat Last 5", "Lucky Star 2 Repeat Last 5",
    "Ball 1 Repeat Last 10", "Ball 2 Repeat Last 10", "Ball 3 Repeat Last 10", "Ball 4 Repeat Last 10", "Ball 5 Repeat Last 10", "Lucky Star 1 Repeat Last 10", "Lucky Star 2 Repeat Last 10",
    "Ball 1 Repeat Last 20", "Ball 2 Repeat Last 20", "Ball 3 Repeat Last 20", "Ball 4 Repeat Last 20", "Ball 5 Repeat Last 20", "Lucky Star 1 Repeat Last 20", "Lucky Star 2 Repeat Last 20"
]
column_indices = {name: i for i, name in enumerate(column_labels)}


df = pd.read_excel("lottery-numbers-tracker.xlsx", sheet_name="EuroMillions", header=0)

df[column_labels].to_csv("euromillions-dataset.csv", index=None)

Num GPUs Available:  0


In [2]:
df = pd.read_csv('euromillions-dataset.csv')
df

,Lucky Star 1,Lucky Star 2,Ball 1,Ball 2,Ball 3,Ball 4,Ball 5,Ball Delta,Ball Y Delta,LS Delta,...,Ball 5 Repeat Last 10,Lucky Star 1 Repeat Last 10,Lucky Star 2 Repeat Last 10,Ball 1 Repeat Last 20,Ball 2 Repeat Last 20,Ball 3 Repeat Last 20,Ball 4 Repeat Last 20,Ball 5 Repeat Last 20,Lucky Star 1 Repeat Last 20,Lucky Star 2 Repeat Last 20
0,2,11,1,3,11,20,43,6.509608,3.888444,9,...,1,1,0,2,4,1,2,1,2,1
1,5,7,5,14,25,26,40,4.993746,2.374868,2,...,0,3,1,1,3,2,1,0,4,3
2,3,10,4,14,31,36,38,5.111262,2.561250,7,...,0,1,2,2,3,0,2,0,1,4
3,1,6,3,21,36,41,46,6.118619,3.698648,5,...,0,1,2,3,2,1,5,1,2,5
4,8,10,10,23,24,29,45,5.309190,4.643275,2,...,0,2,1,2,2,4,6,1,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,6,9,1,19,36,38,49,6.791539,2.814249,3,...,0,0,1,0,1,1,0,0,0,1
312,9,10,10,25,29,34,45,4.918079,3.143247,1,...,0,0,0,0,0,0,0,0,0,0
313,2,4,6,16,18,39,47,6.169481,3.831449,2,...,0,1,0,0,0,0,1,0,1,0
314,7,12,5,14,35,36,39,5.766281,4.422669,5,...,0,0,0,0,0,0,0,0,0,0


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# --- 1. Data Preparation ---

def create_sequences(data, seq_length, pred_cols):
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length)].values[::-1]
        y = data.iloc[i + seq_length, :pred_cols].values
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# --- 2. Load and Preprocess Data ---

# Parameters
seq_length = 7  # Number of past rows to look at
pred_cols = 7  # Number of columns to predict
all_cols = 32
random_seed_length = 7 # Length of random seed

In [4]:
max_values = np.concatenate(([12, 12, 50, 50, 50, 50, 50], np.zeros(all_cols - pred_cols)))
min_values = np.concatenate(([1, 1, 1, 1, 1, 1, 1], np.zeros(all_cols - pred_cols)))

# Scale the data
scaler = MinMaxScaler()
scaled_df = df[::-1]
scaled_df = pd.concat([scaled_df, pd.DataFrame([max_values], columns=df.columns)], ignore_index=True)
scaled_df = pd.concat([scaled_df, pd.DataFrame([min_values], columns=df.columns)], ignore_index=True)
scaled_df.shape

(318, 32)

In [5]:
# Scale the data
scaled_df = pd.DataFrame(scaler.fit_transform(scaled_df), columns=df.columns)
scaled_df = scaled_df.iloc[:-2]

# Create sequences
X, y = create_sequences(scaled_df, seq_length, pred_cols)

# Split into training, validation, and test sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

# Input for the sequence
sequence_input = tf.keras.layers.Input(shape=(seq_length, all_cols))

# Input for the random seed
random_seed_input = tf.keras.layers.Input(shape=(random_seed_length,))

scaled_df.shape

(316, 32)

In [6]:
scaler.inverse_transform([np.concatenate((y[-1], np.zeros(all_cols - pred_cols)))])

array([[ 2., 11.,  1.,  3., 11., 20., 43.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.]])

In [8]:
scaler.inverse_transform(X[-1])

array([[ 5.        ,  7.        ,  5.        , 14.        , 25.        ,
        26.        , 40.        ,  4.99374609,  2.37486842,  2.        ,
         2.54950976,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  1.        ,  0.        ,  2.        ,
         0.        ,  0.        ,  0.        ,  3.        ,  1.        ,
         1.        ,  3.        ,  2.        ,  1.        ,  0.        ,
         4.        ,  3.        ],
       [ 3.        , 10.        ,  4.        , 14.        , 31.        ,
        36.        , 38.        ,  5.11126208,  2.56124969,  7.        ,
         3.16227766,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  2.        ,  1.        ,  1.        ,
         0.        ,  2.        ,  0.        ,  1.        ,  2.        ,
         2.        ,  3.        ,  0.        ,  2.        ,  0.        ,
         1.        ,  4.        ],
       [ 1.        ,  6.        ,  3.        , 21.    

In [9]:
# for testing sequences
# df = df[::-1]
# print(y[-1])
# print(df.iloc[-1].values[:7])

In [10]:
# --- 3. Model Definition ---

# LSTM layer
lstm1_1 = tf.keras.layers.LSTM(128, return_sequences=False)(sequence_input)

dense_1 = tf.keras.layers.Dense(128, activation='sigmoid')(lstm1_1)

dense_2 = tf.keras.layers.Dense(14, activation='sigmoid')(dense_1)

concat = tf.keras.layers.Concatenate()([dense_2, random_seed_input])

output = tf.keras.layers.Dense(pred_cols)(concat)

# Create the model with two inputs
lstm_model = tf.keras.models.Model(inputs=[sequence_input, random_seed_input], outputs=output)

In [11]:

# --- 4. Loss Function with Emphasis ---

def custom_loss(y_true, y_pred, random_target):
    # Calculate loss
    squared_error = tf.square(y_true - y_pred)
    weighted_mse_loss = tf.reduce_mean(squared_error)

    # MSE between prediction and random target
    mse_loss_random = tf.reduce_mean(tf.square(y_pred - random_target))

    # Combine the two losses (you can adjust the weights)
    return 0.6 * weighted_mse_loss + 0.4 * mse_loss_random

def generate_random_target(batch_size):
    targets = []
    for _ in range(batch_size):
        lucky_target = []
        target = []
        while len(lucky_target) < 2:
            num = np.random.randint(1, 13)
            if num not in lucky_target:
                lucky_target.append(num)
        while len(target) < 5:
            num = np.random.randint(1, 51)
            if num not in target:
                target.append(num)
        lucky_target = np.array(lucky_target) / 12
        target = np.array(target) / 50
        targets.append(np.concatenate((lucky_target, target)))
    return np.array(targets)

# --- 5. Compile the Model ---

lstm_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                   loss=custom_loss)  # Use the custom weighted loss


In [12]:
print(X_train.shape)

print(scaler.data_min_)

print(scaler.data_max_)

(278, 7, 32)
[1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
[12.         12.         50.         50.         50.         50.
 50.          8.10092587 11.56719499 11.          9.05538514  2.
  3.          4.          3.          3.          3.          4.
  4.          5.          5.          5.          5.          5.
  6.          6.          6.          7.          6.          7.
  9.         10.        ]


In [13]:
# --- 6. Train the Model ---

# Custom training loop
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

epochs = 300
batch_size = 16

for epoch in range(epochs):
    epoch_loss = 0
    for i in range(0, X_train.shape[0], batch_size):
        # Get batch data

        X_batch = X_train[i:(i + batch_size)]
        y_batch = y_train[i:(i + batch_size)]
        random_seeds_batch = generate_random_target(X_batch.shape[0])

        with tf.GradientTape() as tape:
            # Get predictions
            predictions = lstm_model([X_batch, random_seeds_batch], training=True)

            # Combine the two losses (you can adjust the weights)
            loss = custom_loss(y_batch, predictions, random_seeds_batch)

        # Compute and apply gradients
        gradients = tape.gradient(loss, lstm_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, lstm_model.trainable_variables))

        epoch_loss += loss

    # Print epoch loss
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / (X_train.shape[0] // batch_size)}")



Epoch 1/300, Loss: 0.506454586982727
Epoch 2/300, Loss: 0.17006602883338928
Epoch 3/300, Loss: 0.1321173906326294
Epoch 4/300, Loss: 0.11522729694843292
Epoch 5/300, Loss: 0.10949696600437164
Epoch 6/300, Loss: 0.1073889434337616
Epoch 7/300, Loss: 0.10905632376670837
Epoch 8/300, Loss: 0.09713529050350189
Epoch 9/300, Loss: 0.10160938650369644
Epoch 10/300, Loss: 0.09866175055503845
Epoch 11/300, Loss: 0.10199722647666931
Epoch 12/300, Loss: 0.09698591381311417
Epoch 13/300, Loss: 0.09384238719940186
Epoch 14/300, Loss: 0.09301091730594635
Epoch 15/300, Loss: 0.09540598839521408
Epoch 16/300, Loss: 0.09302066266536713
Epoch 17/300, Loss: 0.08766140788793564
Epoch 18/300, Loss: 0.08967989683151245
Epoch 19/300, Loss: 0.08745458722114563
Epoch 20/300, Loss: 0.08675780147314072
Epoch 21/300, Loss: 0.08424884080886841
Epoch 22/300, Loss: 0.0842650756239891
Epoch 23/300, Loss: 0.07974892109632492
Epoch 24/300, Loss: 0.08194193243980408
Epoch 25/300, Loss: 0.07939905673265457
Epoch 26/300, 

In [14]:
# --- 8. Evaluate the Model ---

val_loss = 0
for i in range(0, X_val.shape[0], batch_size):
    X_batch = X_val[i:i + batch_size]
    y_batch = y_val[i:i + batch_size]
    random_target_batch = generate_random_target(X_batch.shape[0])

    predictions = lstm_model([X_batch, random_target_batch], training=False)
    loss = custom_loss(y_batch, predictions, random_target_batch)
    val_loss += loss

print(f"Validation Loss: {val_loss / (X_val.shape[0] // batch_size)}")

Validation Loss: 0.11377500742673874


In [16]:
# --- 9. Make Predictions ---

# Generate a random seed for prediction
prediction_random_seed = generate_random_target(1)

# Get the last sequence from X_val and reshape it for prediction
last_sequence = X[-1].reshape(1, seq_length, all_cols)

# Make a prediction using the last sequence and the random seed
prediction = lstm_model.predict([last_sequence, prediction_random_seed])

# Inverse transform to get predictions in the original scale
prediction = scaler.inverse_transform(np.concatenate((prediction, np.zeros((prediction.shape[0], all_cols - pred_cols))), axis=1))[:, :pred_cols]

prediction

1/1 [==============================] - 0s 13ms/step


array([[ 4.22921571,  6.27794999, 17.12253898, 17.51735818, 19.63531116,
        14.97618437, 36.4119612 ]])

In [15]:
print(scaler.inverse_transform(X[-1])[:, :pred_cols]);

[[ 2.  7. 22. 30. 41. 44. 49.]
 [ 1.  9.  9. 37. 41. 42. 49.]
 [ 3.  8.  2. 11. 19. 30. 49.]
 [ 4.  5.  6.  8. 14. 27. 41.]
 [ 6.  8.  8. 15. 24. 35. 42.]
 [ 5.  9. 18. 20. 29. 41. 48.]
 [ 6.  7. 12. 27. 36. 37. 42.]]
